In [25]:
import ee
import geemap
import geemap.colormaps as cm

try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


In [26]:
region = ee.Geometry.Polygon([
  [
    [10.7, 50.3],
    [10.7, 50.471],
    [10.9, 50.471],
    [10.9, 50.3],
    [10.7, 50.3] 
  ]
])



In [60]:
def calculate_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    cvi = image.select('B8').multiply(image.select('B4')).rename('CVI')  # Chlorophyll Vegetation Index
    ari = image.select('B3').subtract(image.select('B5')).rename('ARI')  # Anthocyanin Reflectance Index
    msavi = image.expression(
        '((2 * NIR + 1) - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED))) / 2', {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }).rename('MSAVI')  # Modified Soil Adjusted Vegetation Index
    
    return image.addBands([ndvi, cvi, ari, msavi])
    
years = [2017, 2018, 2019, 2020,2021,2022]
indices_images = {}

for year in years:
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(region)
        .filterDate(f'{year}-07-01', f'{year}-09-30')
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
        .map(calculate_indices)
    )
    print(f'Number of images in the collection {year}: ', len(collection.aggregate_array('system:index').getInfo()))
    indices_images[year] = collection.median().clip(region)

indices_diffs = {}
for i in range(len(years) - 1):
    year1, year2 = years[i], years[i + 1]
    indices_diffs[f'{year2}_{year1}'] = indices_images[year2].select(['NDVI', 'CVI', 'ARI', 'MSAVI']).subtract(
        indices_images[year1].select(['NDVI', 'CVI', 'ARI', 'MSAVI'])
    ).rename([f'NDVI_diff_{year2}_{year1}', f'CVI_diff_{year2}_{year1}', 
              f'ARI_diff_{year2}_{year1}', f'MSAVI_diff_{year2}_{year1}'])

nri = (
    indices_diffs['2020_2019'].select('NDVI_diff_2020_2019')
    .add(indices_diffs['2019_2018'].select('NDVI_diff_2019_2018'))
    .add(indices_diffs['2018_2017'].select('NDVI_diff_2018_2017'))
).divide(indices_images[2017].select('NDVI')).rename('NRI')

# Combine all difference layers into a feature stack and add NRI
feature_stack = indices_diffs['2018_2017'].addBands([
    indices_diffs['2019_2018'],
    indices_diffs['2020_2019'],
    nri
])


Number of images in the collection 2017:  5
Number of images in the collection 2018:  21
Number of images in the collection 2019:  14
Number of images in the collection 2020:  13
Number of images in the collection 2021:  13
Number of images in the collection 2022:  6


In [61]:
num_clusters = 3
training = feature_stack.sample(region, scale=20, numPixels=5000)
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(training)
indices_clustered = feature_stack.cluster(clusterer).clip(region)

In [62]:
glyphosate_fields = indices_clustered.eq(0)

In [63]:
palette = cm.get_palette("inferno", num_clusters) 
cluster_vis = {
    'min': 0,
    'max': num_clusters - 1,  
    'palette': palette
}

s2_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

aster_vis = {
    'bands': ['B02', 'B01'],
    'min': 0,
    'max': 255
}
nri_vis = {
    'min': -1,  
    'max': 1,   
    'palette': ['red', 'white', 'green']
}
ndvi_vis = {'min': -0.2, 'max': 0.8, 'palette': ['blue', 'white', 'green']}


Map = geemap.Map()
Map.addLayer(nri, nri_vis, 'NDVI Recovery Index (NRI)')
Map.addLayer(indices_images[2017],s2_vis,'Satelite 2017')
Map.addLayer(indices_images[2018],s2_vis,'Satelite 2018')
Map.addLayer(indices_images[2019],s2_vis,'Satelite 2019')
Map.addLayer(indices_images[2020],s2_vis,'Satelite 2020')
Map.addLayer(indices_images[2021],s2_vis,'Satelite 2021')
Map.addLayer(indices_images[2022],s2_vis,'Satelite 2022')
Map.addLayer(indices_clustered, cluster_vis, 'Field Clusters')
Map.addLayer(glyphosate_fields.updateMask(glyphosate_fields), {'palette': ['red']}, 'Glyphosate Fields')
Map.setCenter(10.8, 50.35, 11)
Map

Map(center=[50.35, 10.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [49]:
unique_clusters = nri_clustered.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=region,
    scale=30,
    maxPixels=1e8
).get('cluster')

# Inspect the unique cluster distribution (for debugging)
print("Unique Cluster Distribution: ", unique_clusters.getInfo())

Unique Cluster Distribution:  {'0': 112830.61960784336, '1': 18273.399999999987, '2': 339868.76470588}


In [67]:
fc = ee.FeatureCollection(Map.draw_features)
region = fc.geometry()
cords = region.getInfo()['coordinates']
cords

[[[10.832477, 50.454643],
  [10.831919, 50.453974],
  [10.825396, 50.453523],
  [10.825481, 50.451993],
  [10.83385, 50.451528],
  [10.836575, 50.453728],
  [10.832627, 50.454657]],
 [[10.83385, 50.450804],
  [10.832541, 50.449643],
  [10.835567, 50.449083],
  [10.837734, 50.44896],
  [10.840094, 50.448126],
  [10.839772, 50.449274],
  [10.838549, 50.449861],
  [10.833871, 50.450763]],
 [[10.832562, 50.464215],
  [10.835127, 50.464475],
  [10.835556, 50.464311],
  [10.834376, 50.462296],
  [10.83267, 50.463539],
  [10.832573, 50.464284]],
 [[10.820503, 50.422199],
  [10.820868, 50.42481],
  [10.821705, 50.427749],
  [10.831575, 50.425316],
  [10.832627, 50.42455],
  [10.831532, 50.424345],
  [10.831361, 50.423675],
  [10.828357, 50.424318],
  [10.823979, 50.420203],
  [10.822885, 50.420791],
  [10.822928, 50.421447],
  [10.822134, 50.421871],
  [10.820568, 50.422199]],
 [[10.827284, 50.414255],
  [10.826962, 50.413312],
  [10.828099, 50.413298],
  [10.828829, 50.413025],
  [10.833786, 